# Dacon 이미지 초해상화
품질이 저하된 저해상도 촬영 이미지(512X512)를 고품질의 고해상도 촬영 이미지(2048X2048)로 생성

<br>
train patch 128*128 : 1640 X 16 = 26240개 patch<br>
test patch 128*128 :  18 X 49 = 882개 patch
<Br>

RRDB  PSNR : 23.2260 <Br>
RRDB+(Self-ensemble) PSNR : 23.4081 <Br>

#Setting

In [ ]:
from google.colab import drive
drive.mount("/content/drive",force_remount = True)

Mounted at /content/drive


In [ ]:
!git clone https://github.com/XPixelGroup/BasicSR.git

Cloning into 'BasicSR'...
remote: Enumerating objects: 5905, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 5905 (delta 0), reused 1 (delta 0), pack-reused 5902
Receiving objects: 100% (5905/5905), 4.15 MiB | 27.60 MiB/s, done.
Resolving deltas: 100% (3729/3729), done.


In [ ]:
!pip install basicsr
!pip install timm
!pip install patchify

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 172 kB 4.9 MB/s 
     |████████████████████████████████| 5.7 MB 59.8 MB/s 
     |████████████████████████████████| 190 kB 78.4 MB/s 
  Created wheel for basicsr: filename=basicsr-1.4.2-py3-none-any.whl size=214839 sha256=9b7927ad59c068e0c620cc1e0b9b3abc525d033b9f547b0858567aa65dbfa64a
  Stored in directory: /root/.cache/pip/wheels/4d/d3/95/e17d0bcdd7dcfb0dbf79db006711e434c42036efbf6695ef7f
Successfully built basicsr
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 549 kB 5.0 MB/s 
     |████████████████████████████████| 182 kB 90.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import random
import pandas as pd
import numpy as np
import os
import sys
import cv2
import zipfile
import math
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.utils import save_image, make_grid
from torch.utils.data import Dataset, DataLoader
from torch.utils import data
from torch.utils.data.sampler import SubsetRandomSampler


from tqdm.auto import tqdm

import torchvision.models as mondels

import warnings
warnings.filterwarnings(action = "ignore")

import numpy as np
from patchify import patchify, unpatchify
from PIL import Image

In [ ]:
# Rotate Library
from basicsr.utils.registry import ARCH_REGISTRY
from basicsr.archs.arch_util import default_init_weights, make_layer, pixel_unshuffle

In [ ]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
device #cuda 할당 확인 

device(type='cuda', index=0)

In [ ]:
# GPU 이름 체크(cuda:0에 연결된 그래픽 카드 기준)
print(torch.cuda.get_device_name()) # 'A100-SXM4-40GB'

# 사용 가능 GPU 개수 체크
print(torch.cuda.device_count()) # 1

Tesla T4
1


## Hyperparameter

In [ ]:
CFG = {
    "RRDB_EPOCHS" : 7,
    "RRDB_LEARNING_RATE" : 1e-4,
    "train_batch" : 4, #2,4
    'test_batch' : 1, 
    "SEED" : 42,
}

##Seed 고정

In [ ]:
def seed_everything(seed):
  random.seed(seed)
  os.environ["PYTHONHASHSEED"] = str(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = True

seed_everything(42) #seed 고정

# Patch 형성

##Patch function

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from patchify import patchify
import tifffile as tiff

def making_patches(img_path,patch_size, train = True, is_lr = True):
  img_name = img_path.split("/")[-1][:-4]
  sample_large_img = cv2.imread(img_path) # 여기서 BGR로 읽고 잘라서 그럼



  patches_img = patchify(sample_large_img, (patch_size,patch_size,3), step  = patch_size) ##patch 형성
  

  for i in range(patches_img.shape[0]):
    for j in range(patches_img.shape[1]):

      #one each patch
      single_patch_img = patches_img[i,j, :,:, :]#patch 한장
      single_patch_img = np.squeeze(single_patch_img, axis = 0)

      #saving on patch
      if train :
        if is_lr:
          lr_file = "/content/drive/MyDrive/patches_new/lr/{}_{}_{}.png".format(img_name, i,j)
          if not os.path.isfile(lr_file):
            cv2.imwrite(lr_file,single_patch_img)
        else:
          hr_file = "/content/drive/MyDrive/patches_new/hr/{}_{}_{}.png".format(img_name, i,j)
          if not os.path.isfile(hr_file):
            cv2.imwrite(hr_file,single_patch_img)

      else:
        test_file ="./patches_new/test_new/{}_{}_{}.png".format(img_name, i,j)
        # test_file ="/content/drive/MyDrive/test_new/{}_{}_{}.png".format(img_name, i,j)
        if not os.path.isfile(test_file):
          cv2.imwrite(test_file,single_patch_img)

  print("processing :", img_name)

##Train patch

In [ ]:
path = "/content/drive/Shareddrives/2022-2 KUBIG CV 이미지초해상도/Data"
os.chdir(path)
train_df = pd.read_csv("./train.csv")
test_df = pd.read_csv("./test.csv")

for i in range(len(train_df)):
  lr_path = train_df["LR"].iloc[i]
  hr_path = train_df["HR"].iloc[i]

  making_patches(lr_path, 128, train = True, is_lr = True)
  making_patches(hr_path, 512 ,train = True, is_lr = False)


#마지막 실행 : "/content/drive/MyDrive/patches_new 에 저장함

In [ ]:
#train patch 생성 : 1640 -> 1640*16(26240), non-overlapping
patch_df=  pd.DataFrame(columns = ["LR", "HR"])
for idx in range(len(train_df)):
  file_name = train_df["LR"][idx].split("/")[-1][:-4]
  for i in range(4):
    for j in range(4):
      temp = pd.DataFrame({"LR" : [f"./patches_new/lr_new/{file_name}_{i}_{j}.png"],
                           "HR" : [f"./patches_new/hr_new/{file_name}_{i}_{j}.png"]})
      patch_df = patch_df.append(temp)
print(len(patch_df))
print(patch_df.head())
patch_df.to_csv(path+"/patch_train.csv", index = False)

## Test patch

In [ ]:
path = "/content/drive/Shareddrives/2022-2 KUBIG CV 이미지초해상도/Data"
os.chdir(path)
test_df = pd.read_csv("./test.csv")

for i in range(len(test_df)):
  test_lr_path = test_df["LR"].iloc[i]

  making_patches(test_lr_path,128, train = False,is_lr = True)

#마지막 실행 : "/content/drive/MyDrive/test_new 에 저장함

In [ ]:
##train patch 생성 : 18 -> 18*49(882), overlapping
patch_test_df=  pd.DataFrame(columns = ["LR"])
for idx in range(len(test_df)):
  file_name = test_df["LR"][idx].split("/")[-1][:-4]
  for i in range(7):
    for j in range(7):
      temp = pd.DataFrame({"LR" : [f"./patches_new/test_new/{file_name}_{i}_{j}.png"]})
      patch_test_df = patch_test_df.append(temp)
print(len(patch_test_df))
print(patch_test_df.head())
patch_test_df.to_csv(path+"/patch_test.csv", index = False)

#Custom Dataset


기본 1640개의 train data, 18개의 test data

*   Train patch: 1장당 균일하게 16등분 (nonoverlapping)
*   lr: 512 -> 128, hr: 2048 -> 512
*   총 1640 * 16 = 26240개 patch



*   Test patch: 1장당 49개의 patch (overlapping)
*   stride = 64 -> (512-128)/64 + 1 = 7
*   7*7 49장의 test patch


train, test 원본은 RGB <br>
train, test patch는 BGR data

경로 설정 잘해야함. 
* 안에 ./train/lr/0000.png 이런식으로 들어가 있어서
* CustomDataset 입력 파라미터 중 base_path를
  * base_path = "/content/drive/Shareddrives/2022-2 KUBIG CV 이미지초해상도/Data" 로 설정해야함

## Training Dataset

train, test 원본은 RGB <br>
train, test patch는 BGR data

In [ ]:
import albumentations as A # for image augmentation
from albumentations.pytorch.transforms import ToTensorV2


In [ ]:
## img to Tensor transforms functions

def get_train_transform():
#   if image_norm == "imagenet": #for pretrained model
#     mean = (0.485, 0.456, 0.406)
#     std = (0.229, 0.224, 0.225)
#   else: #zero-centered Norm (-1~1) <- dataset에서 처음부터 통째로 학습할때
#     mean = (0.5, 0.5, 0.5)
#     std = (0.5, 0.5, 0.5)

  #모두 Non-destructive transformations <- 정보를 추가하거나 손실하지 않게 
  return A.Compose([
      A.RandomRotate90(p=0.7),    
      A.OneOf([
          A.HorizontalFlip(p=1), #수평회전
          A.VerticalFlip(p=1),
          A.ShiftScaleRotate(p=1)],p=0.7), #수직회전
      A.Transpose(p=0.7), #X,Y축 반전
    #   A.Normalize(mean=mean, std=std),  #scaling과 normalize 동시에 처리, Normalize하고 totensor(Albumentation)
    # 하니까 padding 오류남, 255로 그냥 normalize하기로 함 
      ToTensorV2(p = 1.0)],
      additional_targets = {"image" : "image", "label":"image"}
  )       
      

def get_test_transform(): #일반적으로 imagenet 정규화 사용

#   #imagenet
#   mean = (0.485, 0.456, 0.406)
#   std = (0.229, 0.224, 0.225)

  return A.Compose([
    #   A.Normalize(mean=mean, std=std),
      ToTensorV2(p = 1.0)],
      additional_targets ={"image" : "image"}
  )


Epoch 수를 늘리는 것이 곧 여러개의 수많은 이미지를 넣는다는 것과 똑같다 <Br>
즉 데이터 수 자체를 증강하여 1 Epoch에 돌리는 것 = 데이터 transform하여 여러 epoch 돌리는 것 

In [ ]:
class CustomDataset(Dataset):
  def __init__(self, df, transforms, train_mode,base_path, augmentation):
    self.df = df
    self.transforms = transforms #image(png) to Tensor
    self.augmentation = augmentation 
    self.train_mode = train_mode
    self.base_path = base_path

  def __getitem__(self, index): #batch loader
    lr_path = self.df["LR"].iloc[index]
    lr_path = self.base_path +lr_path[1:] 

    lr_img = cv2.imread(lr_path) #load images(BGR)
    
    # lr_img = cv2.cvtColor(lr_img, cv2.COLOR_BGR2RGB)  #BGR to RGB

    
    if self.train_mode:
      #load hr_images as label
      hr_path = self.df["HR"].iloc[index]
      hr_path = self.base_path + hr_path[1:]

      hr_img = cv2.imread(hr_path) #load images(BGR)

      # hr_img = cv2.cvtColor(hr_img, cv2.COLOR_BGR2RGB) #BGR to RGB


      if self.transforms is not None:
        #image to Tensor (with labels)
        transformed = self.transforms(image = lr_img, label = hr_img)
        lr_img = transformed["image"] / 255. # normalization
        hr_img = transformed["label"] / 255.

      # return lr_img, hr_img
      return {'L': lr_img, 'H': hr_img, 'L_path': lr_path, 'H_path': hr_path}

    else:
      #test mode
      #test dataset은 LR 밖에 없다.
      file_name = lr_path.split("/")[-1] ##0000.png

      if self.transforms is not None:
        transformed = self.transforms(image = lr_img)
        lr_img = transformed["image"] / 255.
      
      # return lr_img, file_name
      return {"L" : lr_img, "L_path" : lr_path}

  def __len__(self):
    return len(self.df)

In [ ]:
base_path = "/content/drive/Shareddrives/2022-2 KUBIG CV 이미지초해상도/Data"

train_df = pd.read_csv(base_path +"/patch_train.csv")


#train_df가 우리 원본 파일명들 들어있는 train.csv 파일을 말함
#경로 설정 잘해야함. 
train_dataset = CustomDataset(train_df, get_train_transform(), True, base_path, augmentation = True) #train_mode = true
# rrdb_train_dataset = CustomDataset(train_df, get_train_transform(), True, base_path, augmentation = True) #train_mode = true

valid_size = 0.2


#validation으로 사용할 trainning indices를 얻는다.
num_train = len(train_dataset)
indices = list(range(num_train))
np.random.shuffle(indices) #train data는 shuffle해도 됨
split = int(np.floor(valid_size * num_train)) #np.floor = 가우스 
train_idx, valid_idx = indices[split:], indices[:split]

# trainning, validation batch를 얻기 위한 sampler정의
train_sampler = SubsetRandomSampler(train_idx) #SubsetRandomSampler: 랜덤리스트 
valid_sampler = SubsetRandomSampler(valid_idx)


train_loader = DataLoader(train_dataset, batch_size = CFG["train_batch"],
                          sampler = train_sampler, num_workers = 4,
                          drop_last = True, pin_memory = True)
# rrdb_train_loader = DataLoader(rrdb_train_dataset, batch_size = CFG["train_batch"],
#                           sampler = train_sampler, num_workers = 4,
#                           drop_last = True,pin_memory = False) 

#drop last를 하면 data의 개수가 batch_size로 나눠떨어지지 않을때 마지막 batch 버려준다.
#pin_memory = True : 데이터로더는 Tensor를 CUDA 고정 메모리에 올린다. 
#Dataset이 매우 작을때, 이미 data가 GPU상에 다 올라가 있을 때를 제외하고서는 pin_memory = True가 좋은듯.


valid_loader = DataLoader(train_dataset, batch_size = CFG["train_batch"],
                          sampler = valid_sampler, num_workers = 4,
                          drop_last = True,pin_memory = True)

# rrdb_valid_loader = DataLoader(rrdb_train_dataset, batch_size = CFG["train_batch"],
#                           sampler = valid_sampler, num_workers = 4,
#                           drop_last = True,pin_memory = False)

In [ ]:
# #check shape
# for dict in train_loader:
#   print(dict["L"].shape, dict["H"].shape)   #b * c * H* W 순
#   break

# for dict in valid_loader:
#   print(dict["L"].shape, dict["H"].shape)   #b * c * H* W 순
#   break

## Test Dataset

In [ ]:
#새롭게 만든 7x7 총 18장 * 49 장의 patch들로 만든 dataset
patch_test = pd.read_csv("/content/drive/MyDrive/쿠빅장프/patches_new/patch_test_49.csv")
test_dataset = CustomDataset(patch_test,get_test_transform(), False ,"/content/drive/MyDrive/쿠빅장프", augmentation = False)
test_loader = DataLoader(test_dataset,
                         batch_size = 1,
                         shuffle = False,
                         num_workers = 4,
                         drop_last = False,
                         pin_memory= False)

for dict in test_loader:
  print(dict["L"].shape)   #b * c * H* W 순
  break

torch.Size([1, 3, 128, 128])


#Function

In [ ]:
# from pytorchtools import EarlyStopping
class EarlyStopping:
    """주어진 patience 이후로 validation loss가 개선되지 않으면 학습을 조기 중지"""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt'):
        """
        Args:
            patience (int): validation loss가 개선된 후 기다리는 기간
                            Default: 7
            verbose (bool): True일 경우 각 validation loss의 개선 사항 메세지 출력
                            Default: False
            delta (float): 개선되었다고 인정되는 monitered quantity의 최소 변화
                            Default: 0
            path (str): checkpoint저장 경로
                            Default: 'checkpoint.pt'
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''validation loss가 감소하면 모델을 저장한다.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [ ]:
def train(model, optimizer, scheduler, device, patience, n_epochs):

  model.to(device)
  criterion_1 =  nn.L1Loss().to(device)     #L1 loss 사용, 일반적으로 L2가 PSNR을 최대화하는데 더 좋다고 알려져 있지만, 
  criterion_2 =  nn.MSELoss().to(device)    #L1 loss가 better convergence를 하게 해준다.

  # (L1loss:L2loss = 2:1)


  best_model = None
  best_loss = 9999

  train_losses = []
  valid_losses = []
  avg_train_losses = []
  avg_valid_losses = []  

  # early_stopping object의 초기화
  early_stopping = EarlyStopping(patience = patience, verbose = True)


  for epoch in range(1, n_epochs + 1):


      #########################
      #### train the model ####
      #########################

      model.train()
      # train_loss = []
      for i, img_pair in enumerate(tqdm((train_loader))):
        lr_img, hr_img = img_pair["L"].float().to(device), img_pair["H"].float().to(device)

        optimizer.zero_grad()

        pred_hr_img = model(lr_img)
        loss_1 = criterion_1(pred_hr_img.to(torch.float32), hr_img.to(torch.float32)) #합산 연산하려는 두 배열의 데이터 타입 float32로 통일
        loss_2 = criterion_2(pred_hr_img.to(torch.float32), hr_img.to(torch.float32))

        loss = loss_1 * 2/3 + loss_2 * 1/3  # (L1loss:L2loss = 2:1) <- convergence를 위해 L1에 가중치를 더 주면서도, PSNR 높이기 위해 L2 첨가

        loss.backward()
        optimizer.step() #가중치 갱신 

        train_losses.append(loss.item())

      if scheduler is not None:
        scheduler.step()


      # #epoch 마다 trian loss checking해서 제일 작은 거 선택하기 위함
      # if best_loss > _train_loss:
      #   best_loss = _train_loss
      #   best_model = model

      print("finish the training")

      ######################    
      # validate the model #
      ######################
      model.eval() # prep model for evaluation
      for i, img_pair in enumerate(tqdm((valid_loader))):
        lr_img, hr_img = img_pair["L"].float().to(device), img_pair["H"].float().to(device)

        pred_hr_img = model(lr_img)
        loss_1 = criterion_1(pred_hr_img.to(torch.float32), hr_img.to(torch.float32)) #합산 연산하려는 두 배열의 데이터 타입 float32로 통일
        loss_2 = criterion_2(pred_hr_img.to(torch.float32), hr_img.to(torch.float32))
  
        loss = loss_1 * 2/3 + loss_2 * 1/3
        valid_losses.append(loss.item())

      # print 학습/검증 statistics
      # epoch당 평균 loss 계산
      train_loss = np.average(train_losses)
      valid_loss = np.average(valid_losses)
      avg_train_losses.append(train_loss)
      avg_valid_losses.append(valid_loss)

      epoch_len = len(str(n_epochs))
      batch_len = len(valid_loader)
      
      print_msg = (f'[Epoch {epoch}/{epoch_len}]' +
                   f'[Batch {i}/{batch_len}]' +
                   f'train_loss(L1+L2): {train_loss:.5f} ' +
                   f'valid_loss(L1+L2): {valid_loss:.5f}')
      



      print(print_msg)

      # clear lists to track next epoch
      train_losses = []
      valid_losses = []

      # early_stopping는 validation loss가 감소하였는지 확인이 필요하며,
      # 만약 감소하였을경우 현제 모델을 checkpoint로 만든다.
      early_stopping(valid_loss, model)

      if early_stopping.early_stop:
          print("Early stopping")
          break

  # best model이 저장되어있는 last checkpoint를 로드한다.
  model.load_state_dict(torch.load('checkpoint.pt'))

  return  model, avg_train_losses, avg_valid_losses


In [ ]:
def inference(model, test_loader, evice):
  model.to(device)
  model.eval()

  pred_img_list = []
  name_list = []

  with torch.no_grad() : #for evaluation mode, no gradient operation
    for img_dict in tqdm(test_loader):

      lr_img = img_dict["L"].float().to(device)
      file_name = img_dict["L_path"]

      pred_hr_img = model(lr_img) # batch

      #이미지 format으로 복원
      for pred, name in zip(pred_hr_img, file_name): # batch의 instance별로
        pred = pred.cpu().clone().detach().numpy() #torch -> numpy
        pred = pred.transpose(1,2,0) #channel을 맨 마지막으로  C*H*W - > H*W*C
        pred = pred*255 #다시 pixel값으로 , 정규화해제

        pred_img_list.append(pred.astype("uint8"))
        name_list.append(name)

  return pred_img_list, name_list

In [ ]:
def full_img_from_patches(pred_img_list, pred_name_list): #cropped_number = 한 이미지가 patch로 잘린 개수 ex)4x4 = 16
  pred_img= []
  pred_name = []
  patches_num = len(pred_name_list)

  for i in range(patches_num//49): #원래 test 이미지 개수
    name = pred_name_list[i*49].split("/")[-1].split("_")[0]
    name = "./"+name+".png"
    patches = np.array(pred_img_list[i*49:(i+1)*49]).reshape((7,7,1, 512,512,3))
    reconstructed = unpatchify(patches, (2048, 2048,3))
    # reconstructed = cv2.cvtColor(reconstructed, cv2.COLOR_BGR2RGB) # 색깔 순서 변환 BGR to RGB
    pred_name.append(name)
    pred_img.append(reconstructed)

  return pred_img, pred_name


#patch들이 모두 RGB로 저장되어있음(imread -> imwrite 되어있기 때문에)
#지금 현재 train에서는 imread한 후 학습을 하기 때문에  BGR로 초해상화됨
# test data도 BGR로 초해상화된다.
# 이를 cv.imwrite로 저장하면 RGB로 잘 저장됨.

# RRDB



*   UNET의 성능이 좋지 않을 것 같아 1등의 RRDBNET(X4 upscaling) 사용
*   좀 더 빠른학습을 위해 pretrained된 모델 가져온 후 재학습



In [ ]:
import functools
import torch
import torch.nn as nn
import torch.nn.functional as F


def make_layer_ss(block, n_layers):
    layers = []
    for _ in range(n_layers):
        layers.append(block())
    return nn.Sequential(*layers)


class ResidualDenseBlock_5C(nn.Module):
    """Residual Dense Block.
    Used in RRDB block in ESRGAN.
    Args:
        num_feat (int): Channel number of intermediate features.
        num_grow_ch (int): Channels for each growth.
    """

    def __init__(self, num_feat=64, num_grow_ch=32, bias=True):
        super(ResidualDenseBlock_5C, self).__init__()
        self.conv1 = nn.Conv2d(num_feat, num_grow_ch, 3, 1, 1, bias=bias)
        self.conv2 = nn.Conv2d(num_feat + num_grow_ch, num_grow_ch, 3, 1, 1, bias=bias)
        self.conv3 = nn.Conv2d(num_feat + 2 * num_grow_ch, num_grow_ch, 3, 1, 1, bias=bias)
        self.conv4 = nn.Conv2d(num_feat + 3 * num_grow_ch, num_grow_ch, 3, 1, 1, bias=bias)
        self.conv5 = nn.Conv2d(num_feat + 4 * num_grow_ch, num_feat, 3, 1, 1, bias=bias)

        self.lrelu = nn.LeakyReLU(negative_slope=0.2, inplace=True)

        # initialization
        default_init_weights([self.conv1, self.conv2, self.conv3, self.conv4, self.conv5], 0.1)

    def forward(self, x):
        x1 = self.lrelu(self.conv1(x))
        x2 = self.lrelu(self.conv2(torch.cat((x, x1), 1)))
        x3 = self.lrelu(self.conv3(torch.cat((x, x1, x2), 1)))
        x4 = self.lrelu(self.conv4(torch.cat((x, x1, x2, x3), 1)))
        x5 = self.conv5(torch.cat((x, x1, x2, x3, x4), 1))
        # Empirically, we use 0.2 to scale the residual for better performance
        return x5 * 0.2 + x


class RRDB(nn.Module):
    """Residual in Residual Dense Block.
    Used in RRDB-Net in ESRGAN.
    Args:
        num_feat (int): Channel number of intermediate features.
        num_grow_ch (int): Channels for each growth.
    """

    def __init__(self, num_feat, num_grow_ch=32):
        super(RRDB, self).__init__()
        self.RDB1 = ResidualDenseBlock_5C(num_feat, num_grow_ch)
        self.RDB2 = ResidualDenseBlock_5C(num_feat, num_grow_ch)
        self.RDB3 = ResidualDenseBlock_5C(num_feat, num_grow_ch)

    def forward(self, x):
        out = self.RDB1(x)
        out = self.RDB2(out)
        out = self.RDB3(out)
        # Empirically, we use 0.2 to scale the residual for better performance
        return out * 0.2 + x

class RRDBNet(nn.Module):
    def __init__(self, num_in_ch, num_out_ch, scale=4, num_feat=64, num_block=23, num_grow_ch=32):
        super(RRDBNet, self).__init__()
        RRDB_block_f = functools.partial(RRDB, num_feat=num_feat, num_grow_ch=num_grow_ch)

        self.conv_first = nn.Conv2d(num_in_ch, num_feat, 3, 1, 1, bias=True)
        self.RRDB_trunk = make_layer(RRDB, num_block, num_feat=num_feat, num_grow_ch=num_grow_ch)
        self.trunk_conv = nn.Conv2d(num_feat, num_feat, 3, 1, 1, bias=True)
        # upsample
        self.upconv1 = nn.Conv2d(num_feat, num_feat, 3, 1, 1, bias=True)
        self.upconv2 = nn.Conv2d(num_feat, num_feat, 3, 1, 1, bias=True)
        self.HRconv = nn.Conv2d(num_feat, num_feat, 3, 1, 1, bias=True)
        self.conv_last = nn.Conv2d(num_feat, num_out_ch, 3, 1, 1, bias=True)

        self.lrelu = nn.LeakyReLU(negative_slope=0.2, inplace=True)

    def forward(self, x):
        feat = self.conv_first(feat)
        body_feat = self.trunk_conv(self.RRDB_trunk(feat))
        feat = feat + body_feat
        # upsample
        feat = self.lrelu(self.upconv1(F.interpolate(feat, scale_factor=2, mode='nearest')))
        feat = self.lrelu(self.upconv2(F.interpolate(feat, scale_factor=2, mode='nearest')))
        out = self.conv_last(self.lrelu(self.HRconv(feat)))
        
        return out

@ARCH_REGISTRY.register()
class RRDBNetRot(nn.Module):

    def __init__(self, num_in_ch, num_out_ch, scale=4, num_feat=64, num_block=23, num_grow_ch=32):
        super(RRDBNetRot, self).__init__()
        self.scale = scale
        if scale == 2:
            num_in_ch = num_in_ch * 4
        elif scale == 1:
            num_in_ch = num_in_ch * 16
        self.conv_first = nn.Conv2d(num_in_ch, num_feat, 3, 1, 1, bias=True)
        self.RRDB_trunk = make_layer(RRDB, num_block, num_feat=num_feat, num_grow_ch=num_grow_ch)
        self.trunk_conv = nn.Conv2d(num_feat, num_feat, 3, 1, 1, bias=True)
        # upsample
        self.upconv1 = nn.Conv2d(num_feat, num_feat, 3, 1, 1, bias=True)
        self.upconv2 = nn.Conv2d(num_feat, num_feat, 3, 1, 1, bias=True)
        self.HRconv = nn.Conv2d(num_feat, num_feat, 3, 1, 1, bias=True)
        self.conv_last = nn.Conv2d(num_feat, num_out_ch, 3, 1, 1, bias=True)

        self.lrelu = nn.LeakyReLU(negative_slope=0.2, inplace=True)

    def _forward(self, x):
        if self.scale == 2:
            feat = pixel_unshuffle(x, scale=2)
        elif self.scale == 1:
            feat = pixel_unshuffle(x, scale=4)
        else:
            feat = x
        feat = self.conv_first(feat)
        body_feat = self.trunk_conv(self.RRDB_trunk(feat))
        feat = feat + body_feat
        # upsample
        feat = self.lrelu(self.upconv1(F.interpolate(feat, scale_factor=2, mode='nearest')))
        feat = self.lrelu(self.upconv2(F.interpolate(feat, scale_factor=2, mode='nearest')))
        out = self.conv_last(self.lrelu(self.HRconv(feat)))
        return out
    
    def _get_rotateds(self, x):
        x90 = torch.rot90(x, k=1, dims=(2, 3))
        x180 = torch.rot90(x, k=2, dims=(2, 3))
        x270 = torch.rot90(x, k=3, dims=(2, 3))
        x_flip = torch.flip(x, dims=(3,))
        x_flip90 = torch.rot90(x_flip, k=1, dims=(2, 3))
        x_flip180 = torch.rot90(x_flip, k=2, dims=(2, 3))
        x_flip270 = torch.rot90(x_flip, k=3, dims=(2, 3))
        return (x, x90, x180, x270, x_flip, x_flip90, x_flip180, x_flip270)
       
    def _get_derotateds(self, xs):
        x, x90, x180, x270, x_flip, x_flip90, x_flip180, x_flip270 = xs
        x90 = torch.rot90(x90, k=3, dims=(2, 3))
        x180 = torch.rot90(x180, k=2, dims=(2, 3))
        x270 = torch.rot90(x270, k=1, dims=(2, 3))
        x_flip = torch.flip(x_flip, dims=(3,))
        x_flip90 = torch.rot90(x_flip90, k=3, dims=(2, 3))
        x_flip90 = torch.flip(x_flip90, dims=(3,))
        x_flip180 = torch.rot90(x_flip180, k=2, dims=(2, 3))
        x_flip180 = torch.flip(x_flip180, dims=(3,))
        x_flip270 = torch.rot90(x_flip270, k=1, dims=(2, 3))
        x_flip270 = torch.flip(x_flip270, dims=(3,))
        return (x, x90, x180, x270, x_flip, x_flip90, x_flip180, x_flip270)
        
    def forward(self, x):
        xs = self._get_rotateds(x)
        ys = []
        for x in xs:
            y = self._forward(x)
            ys.append(y)
        ys = self._get_derotateds(ys)
        ys = torch.stack(ys, dim=0)
        y = torch.mean(ys, dim=0)
        return y

## RRDBNet

In [ ]:
rrdb = RRDBNet(num_in_ch = 3, num_feat = 64, num_out_ch = 3, num_block = 23)

ESRGAN 논문에서 23 RRDB blocks를 가지고 높은 PSNR 수치를 보였던 pretrained된 RRDBNET의 가중치 가져온다.

In [ ]:
rrdb.load_state_dict(torch.load("/content/drive/MyDrive/쿠빅장프/models/RRDB/RRDB_PSNR_x4.pth"))
rrdb.to(device)

RRDBNet(
  (conv_first): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (RRDB_trunk): Sequential(
    (0): RRDB(
      (RDB1): ResidualDenseBlock_5C(
        (conv1): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(96, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv3): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv4): Conv2d(160, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv5): Conv2d(192, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (lrelu): LeakyReLU(negative_slope=0.2, inplace=True)
      )
      (RDB2): ResidualDenseBlock_5C(
        (conv1): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(96, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv3): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv4): Conv2d(160, 32, kernel_size=(3, 3), str

### Training

In [ ]:
optimizer = torch.optim.Adam(rrdb.parameters(), lr = CFG["RRDB_LEARNING_RATE"], betas =(0.9,0.999)) #initial decay rates(첫번째, 두번째 moments of gradient)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 3, gamma = 0.5)
best_rrdb, train_loss, valid_loss = train(rrdb, optimizer, scheduler, device, patience = 2, n_epochs = CFG["RRDB_EPOCHS"])
best_rrdb

  0%|          | 0/5248 [00:00<?, ?it/s]

finish the training


  0%|          | 0/1312 [00:00<?, ?it/s]

[Epoch 1/1][Batch 1311/1312]train_loss(L1+L2): 0.03982 valid_loss(L1+L2): 0.03921
Validation loss decreased (inf --> 0.039208).  Saving model ...


  0%|          | 0/5248 [00:00<?, ?it/s]

...

  0%|          | 0/5248 [00:00<?, ?it/s]

finish the training


  0%|          | 0/1312 [00:00<?, ?it/s]

[Epoch 7/1][Batch 1311/1312]train_loss(L1+L2): 0.03767 valid_loss(L1+L2): 0.03791
Validation loss decreased (0.037976 --> 0.037912).  Saving model ...


RRDBNet(
  (conv_first): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (RRDB_trunk): Sequential(
    (0): RRDB(
      (RDB1): ResidualDenseBlock_5C(
        (conv1): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(96, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv3): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv4): Conv2d(160, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv5): Conv2d(192, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (lrelu): LeakyReLU(negative_slope=0.2, inplace=True)
      )
      (RDB2): ResidualDenseBlock_5C(
        (conv1): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(96, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv3): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv4): Conv2d(160, 32, kernel_size=(3, 3), str

In [ ]:
optimizer = torch.optim.Adam(rrdb.parameters(), lr = CFG["RRDB_LEARNING_RATE"], betas =(0.9,0.999)) #initial decay rates(첫번째, 두번째 moments of gradient)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 3, gamma = 0.5)
best_rrdb, train_loss, valid_loss = train(rrdb, optimizer, scheduler, device, patience = 2, n_epochs = CFG["RRDB_EPOCHS"])
best_rrdb

### RRDBNet Model Save

In [ ]:
import joblib

saving_path = "/content/drive/MyDrive/쿠빅장프/patches_new/models/"
os.makedirs(saving_path, exist_ok = True)
joblib.dump(best_rrdb, f'{saving_path}RRDB_patches_epochs:{CFG["RRDB_EPOCHS"]}_lr:{CFG["RRDB_LEARNING_RATE"]}_batch:{CFG["train_batch"]}.pkl')

가중치는 따라 pt 확장자로 저장

In [ ]:
torch.save(rrdb.state_dict(),'rrdb_state_dict.pt')

## RRDB+

In [ ]:
rrdbRot = RRDBNetRot(num_in_ch = 3, num_out_ch= 3, num_feat = 64, num_block = 23, num_grow_ch = 32, scale = 4) #basic 16 blocks, deeper is 23 blocks
# 문제 상황에 맞게 맞춰주기

In [ ]:
rrdbRot.load_state_dict(torch.load("/content/drive/MyDrive/쿠빅장프/models/rrdb_state_dict.pt")) # 가중치 불러온다.
rrdbRot.to(device)

RRDBNetRot(
  (conv_first): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (RRDB_trunk): Sequential(
    (0): RRDB(
      (RDB1): ResidualDenseBlock_5C(
        (conv1): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(96, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv3): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv4): Conv2d(160, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv5): Conv2d(192, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (lrelu): LeakyReLU(negative_slope=0.2, inplace=True)
      )
      (RDB2): ResidualDenseBlock_5C(
        (conv1): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(96, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv3): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv4): Conv2d(160, 32, kernel_size=(3, 3), 

### RRDBRot Model Save

In [ ]:
import joblib

saving_path = "/content/drive/MyDrive/쿠빅장프/patches_new/models/"
os.makedirs(saving_path, exist_ok = True)
joblib.dump(best_rrdb, f'{saving_path}RRDBRot_patches_epochs:{CFG["RRDB_EPOCHS"]}_lr:{CFG["RRDB_LEARNING_RATE"]}_batch:{CFG["train_batch"]}.pkl')

#Inference

In [ ]:
pred_img_list, pred_name_list = inference(rrdbRot, test_loader, device) #18*49 = 882개 image

  0%|          | 0/882 [00:00<?, ?it/s]

In [ ]:
print(pred_img_list[0].shape, pred_name_list[0])

(512, 512, 3) /content/drive/MyDrive/쿠빅장프/patches_new/test_new_new/20000_0_0.png


## Patch merge

patch 붙이기 : patch_size와 stride가 다른 경우

도화지(np.zeros)에 predicted patches를 하나씩 붙인다(더한다.) 해당 posiztion mark(1)를 별도로 저장

이후, 전체 이미지 / mask

고려할 점 : 이미지가 2d가 아니라 3차원임(RGB channel 3) <br>
그냥 unpatchify 써도 될 듯 (겹치지만 어차피 연속적인 patch임)

In [ ]:
pred_img, pred_name = full_img_from_patches(pred_img_list, pred_name_list)

## Reconstruction image saving

In [ ]:
saving_path = "/content/drive/MyDrive/쿠빅장프/images/"
os.makedirs(saving_path + "RRDBRot_V2", exist_ok = True) #해당 디렉토리가 기존에 존재하면 에러없이 넘어감, 없으면 생성 

sub_imgs = []

for path, pred_img in tqdm(zip(pred_name, pred_img)):
  name_idx = path.rfind("/")
  file_name = path[name_idx : ]
  file_path = saving_path + "RRDBRot_V2" + file_name
  cv2.imwrite(file_path, pred_img) # 픽셀값으로 이미지 형성
  sub_imgs.append(file_path)

print("Done")

0it [00:00, ?it/s]

Done


## Dacon Submission

In [ ]:
os.getcwd()

'/content/drive/MyDrive/쿠빅장프/patches_new/submission'

In [ ]:
saving_path = '/content/drive/MyDrive/쿠빅장프/images/RRDBRot_V2'
os.makedirs(saving_path, exist_ok = True)
os.chdir(saving_path)
os.getcwd()

sub_imgs = []

for path, img in tqdm(zip(pred_name, pred_img)):
  cv2.imwrite(path, img) # 픽셀값으로 이미지 형성
  sub_imgs.append(path)
  print(path)
submission = zipfile.ZipFile("submission_RRDBRot.zip", "w")# 압축파일 형성

for path in sub_imgs:
  submission.write(path)#압축파일에 이미지들 추가

submission.close()
print("Done")

0it [00:00, ?it/s]

./20000.png
./20001.png
./20002.png
./20003.png
./20004.png
./20005.png
./20006.png
./20007.png
./20008.png
./20009.png
./20010.png
./20011.png
./20012.png
./20013.png
./20014.png
./20015.png
./20016.png
./20017.png
Done
